In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import pandas as pd
import feather
import os
from six.moves import range

D:\Programming\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.enable_eager_execution()

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 1.11.0
Eager execution: True


In [3]:
data_root ='D:\\Programming\\Projects\\notMNIST'
image_size=28

def feather_3d(file,image_size = 28):
    temp = feather.read_dataframe(file)
    width = np.shape(temp.values)[1]
    if width == 1:
        final = temp.values
        final = final.reshape(-1)
    else:
        final = np.reshape(temp.values, (-1, image_size, image_size))
    return final

def read_data(dir, filename):
    file = os.path.join(dir, filename + '.feather')
    output = feather_3d(file)
    return output

test_dataset = read_data(data_root, 'test_dataset_final')
test_labels = read_data(data_root, 'test_labels_final')
train_dataset = read_data(data_root, 'train_dataset_final')
train_labels = read_data(data_root, 'train_labels_final')
valid_dataset = read_data(data_root, 'valid_dataset_final')
valid_labels = read_data(data_root, 'valid_labels_final')

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (197309, 28, 28) (197309,)
Validation set (17460, 28, 28) (17460,)
Test set (10202, 28, 28) (10202,)


In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale


def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (197309, 28, 28, 1) (197309, 10)
Validation set (17460, 28, 28, 1) (17460, 10)
Test set (10202, 28, 28, 1) (10202, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
num_steps = 1001

with tf.Session(graph=graph,config=config) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.784596
Minibatch accuracy: 0.0%
Validation accuracy: 10.2%
Minibatch loss at step 50: 1.410363
Minibatch accuracy: 68.8%
Validation accuracy: 37.2%
Minibatch loss at step 100: 0.788628
Minibatch accuracy: 68.8%
Validation accuracy: 64.7%
Minibatch loss at step 150: 1.762141
Minibatch accuracy: 56.2%
Validation accuracy: 74.3%
Minibatch loss at step 200: 1.074871
Minibatch accuracy: 62.5%
Validation accuracy: 77.0%
Minibatch loss at step 250: 0.880387
Minibatch accuracy: 68.8%
Validation accuracy: 77.2%
Minibatch loss at step 300: 0.776801
Minibatch accuracy: 62.5%
Validation accuracy: 76.9%
Minibatch loss at step 350: 0.379259
Minibatch accuracy: 87.5%
Validation accuracy: 78.4%
Minibatch loss at step 400: 0.295038
Minibatch accuracy: 93.8%
Validation accuracy: 78.7%
Minibatch loss at step 450: 0.239996
Minibatch accuracy: 93.8%
Validation accuracy: 78.7%
Minibatch loss at step 500: 1.349360
Minibatch accuracy: 68.8%
Validation accuracy: 78.8%
Mi

2